<a href="https://colab.research.google.com/github/Jaquelinedops/Benchmark_image_stegnography/blob/main/blind_watermark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install blind-watermark

In [ ]:
!pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11418 sha256=cfd6dbcd3e2bd6828b3b4aed2bc3bab1511638b6448d348ab457339de7ca95b6
  Stored in directory: /root/.cache/pip/wheels/87/f9/d9/6ec7e7b470df5ba0e317c6988ba7677a124ab220f5effea702
Successfully built sewar


In [ ]:
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sewar.full_ref import vifp # Only importing vifp now
import os
import time
import cv2
import torch

print("Is CUDA available?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Total VRAM:", round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2), "GB")
else:
    print("CUDA not available. GPU not detected.")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#from imwatermark import WatermarkEncoder, WatermarkDecoder
#from trustmark import TrustMark # Assuming TrustMark is installed and accessible
import matplotlib.pyplot as plt


def calculate_mse(image_path1, image_path2):
    """
    Calculates the Mean Squared Error (MSE) between two images given their paths.

    Args:
        image_path1 (str): Path to the first image.
        image_path2 (str): Path to the second image.

    Returns:
        float: The MSE value, or None if an error occurs.
    """
    try:
        img1 = Image.open(image_path1).convert('RGB')
        img2 = Image.open(image_path2).convert('RGB')
    except FileNotFoundError:
        print(f"Error: One or both image files not found: {image_path1}, {image_path2}")
        return None
    except Exception as e:
        print(f"An error occurred while opening images: {e}")
        return None

    np_img1 = np.array(img1)
    np_img2 = np.array(img2)

    if np_img1.shape != np_img2.shape:
        print("Error: Images must have the same dimensions for MSE calculation.")
        return None

    err = np.sum((np_img1.astype("float") - np_img2.astype("float")) ** 2)
    err /= float(np_img1.shape[0] * np_img1.shape[1])
    return err

def calculate_ssim(image_path1, image_path2):
    """
    Calculates the Structural Similarity Index (SSIM) between two images
    given their paths. Images are converted to grayscale for SSIM calculation.

    Args:
        image_path1 (str): Path to the first image.
        image_path2 (str): Path to the second image.

    Returns:
        float: The SSIM value, or None if an error occurs.
    """
    try:
        img1 = Image.open(image_path1).convert('L')  # Convert to grayscale
        img2 = Image.open(image_path2).convert('L')  # Convert to grayscale
    except FileNotFoundError:
        print(f"Error: One or both image files not found: {image_path1}, {image_path2}")
        return None
    except Exception as e:
        print(f"An error occurred while opening images: {e}")
        return None

    np_img1 = np.array(img1)
    np_img2 = np.array(img2)

    if np_img1.shape != np_img2.shape:
        print("Error: Images must have the same dimensions for SSIM calculation.")
        return None

    ssim_value = ssim(np_img1, np_img2)
    return ssim_value



# --- Novas funções para FSIM e VIF ---
def calculate_fsim(image_path1, image_path2):
    """
    Calculates the Feature Similarity Index (FSIM) between two images given their paths.
    Requires images to be in RGB format.
    FSIM typically ranges from 0 to 1, where 1 indicates perfect similarity.
    """
    try:
        img1 = Image.open(image_path1).convert('RGB')
        img2 = Image.open(image_path2).convert('RGB')
    except FileNotFoundError:
        print(f"Error: One or both image files not found for FSIM: {image_path1}, {image_path2}")
        return None
    except Exception as e:
        print(f"An error occurred while opening images for FSIM: {e}")
        return None

    np_img1 = np.array(img1)
    np_img2 = np.array(img2)

    if np_img1.shape != np_img2.shape:
        print("Error: Images must have the same dimensions for FSIM calculation.")
        return None

    # fsim function in sewar expects images as NumPy arrays
    # It returns FSIM value and the Phase Congruency (PC) map, we only need the FSIM value
    fsim_value, _ = sewar.full_ref.fsim(np_img1, np_img2) # full=True makes it return PC map too, but not needed for value
    return fsim_value


def calculate_vif(image_path1, image_path2):
    """
    Calculates the Visual Information Fidelity (VIFp) between two images given their paths.
    Requires images to be in RGB format.
    VIF values are typically non-negative, with higher values indicating better visual fidelity.
    """
    try:
        img1 = Image.open(image_path1).convert('RGB')
        img2 = Image.open(image_path2).convert('RGB')
    except FileNotFoundError:
        print(f"Error: One or both image files not found for VIF: {image_path1}, {image_path2}")
        return None
    except Exception as e:
        print(f"An error occurred while opening images for VIF: {e}")
        return None

    np_img1 = np.array(img1)
    np_img2 = np.array(img2)

    if np_img1.shape != np_img2.shape:
        print("Error: Images must have the same dimensions for VIF calculation.")
        return None

    vif_value = vifp(np_img1, np_img2)
    return vif_value

def get_similarity_metrics(technique_name, image_path1, image_path2):
    """
    Calculates and returns a tuple of (technique_name, mse_value, ssim_value, fsim_value, vif_value)
    for two given image paths. Prints details about the comparison.
    NOTE: FSIM will be None as it's not directly available in used libraries.
    """
    print(f"\n--- Calculando métricas para: {technique_name} entre '{image_path1}' e '{image_path2}' ---")

    mse_value = calculate_mse(image_path1, image_path2)
    ssim_value = calculate_ssim(image_path1, image_path2)
    fsim_value = None # Set FSIM to None, as it's not directly calculated here
    vif_value = calculate_vif(image_path1, image_path2)

    print(f"  MSE: {mse_value:.4f}" if mse_value is not None else "  MSE: N/A")
    print(f"  SSIM: {ssim_value:.4f}" if ssim_value is not None else "  SSIM: N/A")
    print(f"  FSIM: {fsim_value:.4f}" if fsim_value is not None else "  FSIM: N/A (Not available in this setup)")
    print(f"  VIF: {vif_value:.4f}" if vif_value is not None else "  VIF: N/A")
    print("-" * 60)

    print("\n--- Interpretação das Métricas Perceptuais ---")
    print("Essas métricas buscam correlacionar-se melhor com a percepção humana da qualidade e similaridade da imagem:")
    print("- **SSIM**: Excelente para capturar a estrutura, contraste e luminância.")
    print("- **FSIM**: Foca em características de baixo nível que o sistema visual humano é bom em detectar (gradientes de fase e magnitude).")
    print("- **VIFp**: Modela como a informação é extraída pelo sistema visual humano, avaliando a fidelidade da informação visual.")
    print("=" * 70)

    return (technique_name, mse_value, ssim_value, fsim_value, vif_value)

def compare_images(image_path1, image_path2, tecnique_name="Your_model"):
    """
    Compares two images using MSE and SSIM, and prints the results with interpretation.

    Args:
        image_path1 (str): Path to the first image.
        image_path2 (str): Path to the second image.
    """
    print(f"Comparing '{image_path1}' and '{image_path2}':")
    print("-" * 30)
    # Calculate MSE
    mse_value = calculate_mse(image_path1, image_path2)
    if mse_value is not None:
        print(f"Mean Squared Error (MSE): {mse_value:.2f}")
    else:
        print("Could not calculate MSE due to an error.")

    # Calculate SSIM
    ssim_value = calculate_ssim(image_path1, image_path2)
    if ssim_value is not None:
        print(f"Structural Similarity Index (SSIM): {ssim_value:.2f}")
    else:
        print("Could not calculate SSIM due to an error.")

 # Calculate FSIM
    fsim_value = calculate_fsim(image_path1, image_path2)
    if fsim_value is not None:
        print(f"Feature Similarity Index (FSIM): {fsim_value:.4f}")
    else:
        print("  * Não foi possível calcular o FSIM devido a um erro.")

    print("-" * 60)

    # Calculate VIF
    vif_value = calculate_vif(image_path1, image_path2)
    if vif_value is not None:
        print(f"Visual Information Fidelity (VIFp): {vif_value:.4f}")
    else:
        print("  * Não foi possível calcular o VIFp devido a um erro.")

    return (tecnique_name, mse_value, ssim_value, fsim_value, vif_value)

def print_metrics_table():
    """
    Imprime uma tabela explicativa sobre as métricas de similaridade de imagem,
    indicando o que medem, o valor ideal e sua interpretação.
    """
    print("--- Entendendo as Métricas de Similaridade de Imagem ---")
    print("\nAo avaliar a similaridade entre duas imagens, especialmente após processos como compressão ou esteganografia, usamos métricas para quantificar o quão parecidas elas são. Algumas dessas métricas focam na diferença exata de pixels, enquanto outras tentam imitar a percepção humana.\n")

    # Cabeçalho da tabela
    header = f"{'Métrica':<10} | {'O que mede':<50} | {'Valor ideal':<15} | {'Interpretação do valor':<50}"
    separator = "-" * len(header)

    print(header)
    print(separator)

    # Dados das métricas
    metrics_data = [
        ("MSE", "Erro médio quadrático (diferença pixel a pixel)", "Menor", "0 = Imagens idênticas. Maior = Menos similar."),
        ("SSIM", "Similaridade estrutural, contraste, luminância (perceptual)", "Maior", "1 = Perfeitamente similar (perceptual). Próximo de 1 = Alta similaridade."),
        ("VIFp", "Fidelidade da informação visual (perceptual)", "Maior", "Quanto maior, mais informação visual da original foi retida."),
        ("FSIM", "Similaridade de características visuais (perceptual)", "Maior", "1 = Similaridade de características perfeita. Próximo de 1 = Alta similaridade.")
    ]

    # Imprimir as linhas da tabela
    for metric, description, ideal_value, interpretation in metrics_data:
        # Quebrar a descrição se for muito longa para manter o alinhamento
        # Isso é uma simplificação; para quebras de linha complexas, seria necessário um algoritmo mais robusto
        if len(description) > 50:
            desc_lines = [description[i:i+50] for i in range(0, len(description), 50)]
            print(f"{metric:<10} | {desc_lines[0]:<50} | {ideal_value:<15} | {interpretation:<50}")
            for i in range(1, len(desc_lines)):
                print(f"{'':<10} | {desc_lines[i]:<50} | {'':<15} | {'':<50}")
        else:
            print(f"{metric:<10} | {description:<50} | {ideal_value:<15} | {interpretation:<50}")

    print(separator)
    print("\nEscolher a métrica certa depende do seu objetivo. Para qualidade perceptual, **SSIM**, **VIFp** e **FSIM** são geralmente mais relevantes que o **MSE**.")



def display_image_if_exists(image_path, title="Imagem"):
    """
    Attempts to load and display an image at the given path using matplotlib.
    If the image file does not exist, it prints a message and does nothing else.

    Args:
        image_path (str): The full path to the image file (e.g., 'path/to/my_image.png').
        title (str, optional): The title to display above the image. Defaults to "Imagem".
    """
    if not os.path.exists(image_path):
        print(f"Erro: A imagem '{image_path}' não foi encontrada. Não é possível exibir.")
        return

    # Visualize the image
    try:
        current_img = Image.open(image_path)
        plt.figure(figsize=(6, 6))
        plt.imshow(current_img)
        plt.title(title)
        plt.axis('off') # Hide axes for cleaner image display
        plt.show()
    except Exception as e:
        print(f"Não foi possível carregar ou exibir a imagem '{image_path}': {e}")


import os
from PIL import Image, ImageChops # ImageChops for color operations
import matplotlib.pyplot as plt
#from trustmark import TrustMark # Assuming TrustMark is installed and accessible


# --- Helper Function to Display Images (from previous interactions) ---
def display_image_if_exists(image_path, title="Imagem"):
    if not os.path.exists(image_path):
        print(f"Erro: A imagem '{image_path}' não foi encontrada. Não é possível exibir.")
        return

    try:
        current_img = Image.open(image_path)
        plt.figure(figsize=(6, 6))
        plt.imshow(current_img)
        plt.title(title)
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Não foi possível carregar ou exibir a imagem '{image_path}': {e}")

# --- Attack Functions ---

def attack_resize(input_pil_image, output_path, percentage):
    """Resizes the image by a given percentage."""
    width, height = input_pil_image.size
    new_width = int(width * percentage / 100)
    new_height = int(height * percentage / 100)
    resized_img = input_pil_image.resize((new_width, new_height), Image.LANCZOS) # LANCZOS for high quality downsampling
    resized_img.save(output_path)
    return resized_img

def attack_rotate(input_pil_image, output_path, angle):
    """Rotates the image by a given angle."""
    # Expand=True to ensure the entire rotated image is visible,
    # fillcolor for empty areas (black for RGB)
    rotated_img = input_pil_image.rotate(angle, expand=True, fillcolor=(0,0,0) if input_pil_image.mode == 'RGB' else 0)
    rotated_img.save(output_path)
    return rotated_img

def attack_cut(input_pil_image, output_path, percentage):
    """Cuts (crops) a percentage from each side of the image."""
    width, height = input_pil_image.size

    # Calculate crop amount for each side (e.g., 15% means 15% from left, 15% from top etc.)
    crop_w = int(width * percentage / 100)
    crop_h = int(height * percentage / 100)

    # Define the bounding box for the crop (left, upper, right, lower)
    left = crop_w
    upper = crop_h
    right = width - crop_w
    lower = height - crop_h

    if right <= left or lower <= upper:
        print(f"Aviso: Corte de {percentage}% resultaria em imagem vazia ou inválida. Pulando ataque de corte.")
        return None # Return None if crop would make image invalid

    cropped_img = input_pil_image.crop((left, upper, right, lower))
    cropped_img.save(output_path)
    return cropped_img

def attack_black_border(input_pil_image, output_path, border_size_percentage):
    """Adds a black border around the image."""
    width, height = input_pil_image.size
    border_pixels_w = int(width * border_size_percentage / 100)
    border_pixels_h = int(height * border_size_percentage / 100)

    new_width = width + 2 * border_pixels_w
    new_height = height + 2 * border_pixels_h

    # Create a new image with black background
    bordered_img = Image.new(input_pil_image.mode, (new_width, new_height), color='black')

    # Paste the original image into the center
    bordered_img.paste(input_pil_image, (border_pixels_w, border_pixels_h))
    bordered_img.save(output_path)
    return bordered_img

def attack_change_colors_grayscale(input_pil_image, output_path):
    """Converts the image to grayscale."""
    grayscale_img = input_pil_image.convert('L') # 'L' mode for grayscale
    grayscale_img.save(output_path)
    return grayscale_img

def attack_change_colors_invert(input_pil_image, output_path):
    """Inverts the colors of the image."""
    inverted_img = ImageChops.invert(input_pil_image)
    inverted_img.save(output_path)
    return inverted_img


def attack_compress_tiff(input_pil_image, output_path, compression='tiff_deflate'):
    """
    Comprime a imagem para o formato TIFF com uma compressão específica.
    Common compressions: 'raw', 'tiff_deflate', 'jpeg', 'tiff_lzw', 'packbits', 'tiff_ccitt'
    """
    try:
        # TIFF supports various compressions, 'tiff_deflate' (ZIP) is common for lossless
        # 'jpeg' can be used for lossy, but usually better to use direct JPEG compression
        tiff_img = input_pil_image.save(output_path, compression=compression)
        # Reloading to ensure the saved file is read back for consistency
        return Image.open(output_path)
    except Exception as e:
        print(f"Erro ao comprimir para TIFF com compressão '{compression}': {e}")
        return None

def attack_compress_jpeg(input_pil_image, output_path, quality=90):
    """
    Comprime a imagem para o formato JPEG com uma qualidade específica.
    Qualidade varia de 0 (pior) a 100 (melhor).
    """
    try:
        jpeg_img = input_pil_image.save(output_path, quality=quality, optimize=True)
        return Image.open(output_path)
    except Exception as e:
        print(f"Erro ao comprimir para JPEG com qualidade {quality}: {e}")
        return None

def attack_compress_jpeg2000(input_pil_image, output_path, quality_mode='rates', quality=10):
    """
    Comprime a imagem para o formato JPEG2000.
    quality_mode: 'rates' (e.g., quality=5 for 5:1 compression) or 'dB' (e.g., quality=30 for 30dB PSNR).
    """
    try:
        # JPEG2000 (JP2) saving requires specific parameters.
        # Ensure the output path ends with .jp2 or .jpx
        if not output_path.lower().endswith(('.jp2', '.jpx')):
            output_path = os.path.splitext(output_path)[0] + '.jp2'
            print(f"Aviso: Extensão de saída para JPEG2000 ajustada para '{output_path}'.")

        jpeg2000_img = input_pil_image.save(output_path, quality_mode=quality_mode, quality=quality)
        return Image.open(output_path)
    except Exception as e:
        print(f"Erro ao comprimir para JPEG2000 (mode={quality_mode}, quality={quality}): {e}")
        print("Verifique se você tem suporte a JPEG2000 no Pillow (libimageopenjpeg geralmente é necessário).")
        return None

def attack_compress_tiff(input_pil_image, output_path, compression='tiff_deflate'):
    """
    Comprime a imagem para o formato TIFF com uma compressão específica.
    Common compressions: 'raw', 'tiff_deflate', 'jpeg', 'tiff_lzw', 'packbits', 'tiff_ccitt'
    """
    try:
        # TIFF supports various compressions, 'tiff_deflate' (ZIP) is common for lossless
        # 'jpeg' can be used for lossy, but usually better to use direct JPEG compression
        tiff_img = input_pil_image.save(output_path, compression=compression)
        # Reloading to ensure the saved file is read back for consistency
        return Image.open(output_path)
    except Exception as e:
        print(f"Erro ao comprimir para TIFF com compressão '{compression}': {e}")
        return None

# --- Embed Functions ---
def embed_watermark_imwatermark(input_image_path: str, secret_message: str, output_watermarked_image_path: str, alg_name='dwtDct') -> tuple:
    """
    Embeds a watermark into an image using the imwatermark library.

    Args:
        input_image_path (str): Path to the original cover image.
        secret_message (str): The secret message to embed.
        output_watermarked_image_path (str): Path to save the watermarked image.
        alg_name (str): The watermarking algorithm to use ('dwtDct' or 'dctDwt').

    Returns:
        tuple: (watermarked_image_path, encoder, decoder) or (None, None, None) if embedding fails.
               encoder and decoder are initialized instances for future use.
    """
    print(f"\n--- Embedding watermark using imwatermark ({alg_name}) ---")
    print(f"Original image: {input_image_path}")
    print(f"Secret message: '{secret_message}'")

    image = cv2.imread(input_image_path)

    if image is None:
        print(f"Error: Could not load image from {input_image_path}. Embedding failed.")
        return None, None, None

    try:
        start_time = time.time()
        encoder = WatermarkEncoder()
        encoder.set_watermark('bytes', secret_message.encode('utf-8'))
        watermarked_image = encoder.encode(image, alg=alg_name)
        cv2.imwrite(output_watermarked_image_path, watermarked_image)
        end_time = time.time()
        encode_duration = end_time - start_time

        # Initialize decoder with the same watermark type and length
        decoder = WatermarkDecoder('bytes', len(secret_message))

        print(f"Embedding duration: {encode_duration:.4f} seconds")

        return output_watermarked_image_path, encoder, decoder, encode_duration

    except Exception as e:
        print(f"An error occurred during imwatermark embedding: {e}")
        return None, None, None, None


# --- Helper function for invisible-watermark decoding ---


# --- Updated run_attack_test function ---
def run_attack_test(
    attack_name,
    attack_function,
    input_image_path,
    output_image_suffix,
    secret,
    technique=4, # 1 for TrustMark, 2 for invisible-watermark
    decoder=None,
    len_wm : int = 8,
    **attack_kwargs
):
    """
    Runs a specific attack on the watermarked image, attempts to decode,
    and reports the result.

    Args:
        attack_name (str): A descriptive name for the attack.
        attack_function (callable): The function that performs the attack.
                                     It should accept (input_pil_image, output_path, **kwargs)
                                     and return the attacked PIL Image (or None if failed).
        input_image_path (str): Path to the original watermarked image to be attacked.
        output_image_suffix (str): Suffix for the output filename (e.g., '_resized_25.png').
        decoder: The initialized decoder instance (TrustMark or WatermarkDecoder).
        secret (str): The original secret message.
        technique (int): 1 for TrustMark, 2 for invisible-watermark.
        **attack_kwargs: Additional keyword arguments to pass to the attack_function.

    Returns:
        dict: A dictionary with attack results (detected_secret, secret_match, attack_successful).
    """
    print(f"\n===== Testando Ataque: {attack_name} =====")

    # Base name without extension for generating attacked image path
    base_name_no_ext = os.path.splitext(input_image_path)[0]
    attacked_output_path = f"{base_name_no_ext}{output_image_suffix}"

    results = {
        'attack_name': attack_name,
        'attack_successful': False,
        'detected_secret': None,
        'secret_match': False,
        'watermark_present': False,
        'output_image_suffix': output_image_suffix # Storing this for cleanup later
    }

    original_watermarked_pil = None
    try:
        original_watermarked_pil = Image.open(input_image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Erro: Imagem de entrada '{input_image_path}' não encontrada. Abortando teste para '{attack_name}'.")
        return results
    except Exception as e:
        print(f"Erro ao carregar imagem para ataque '{attack_name}': {e}")
        return results

    # Apply the attack
    print(f"Aplicando ataque e salvando em: {attacked_output_path}")
    attacked_image_pil = attack_function(original_watermarked_pil, attacked_output_path, **attack_kwargs)

    if attacked_image_pil is None:
        print(f"Ataque '{attack_name}' falhou ou resultou em imagem inválida. Pulando detecção.")
        return results

    results['attack_successful'] = True

    # Attempt to decode the watermark
    print("--- Tentando decodificar a marca d'água ---")

    if technique == 1:
        # TrustMark decoding
        # Assuming decode_and_evaluate_trustmark exists and works with PIL images and a TrustMark decoder
        results.update(decode_and_evaluate_trustmark(attacked_image_pil, decoder, secret))
    #elif technique == 2:
        # invisible-watermark decoding
        # The algorithm name should be passed to the helper, assuming it's 'dwtDct' for this context.
        #results.update(decode_and_evaluate_imwatermark(image_path=attacked_output_path, original_secret_message=secret, algorithm='dwtDct'))
    #elif technique == 3:
        # invisible-watermark decoding
        # The algorithm name should be passed to the helper, assuming it's 'dctDwt' for this context.
        #results.update(decode_and_evaluate_imwatermark(image_path=attacked_output_path, original_secret_message=secret, algorithm='dctDwt'))
    elif technique == 4:
        # invisible-watermark decoding
        # The algorithm name should be passed to the helper, assuming it's 'dctDwt' for this context.
        results.update(decode_and_evaluate_watermark(image_path=attacked_output_path, original_secret_message=SECRET_MESSAGE,len_wm = len_wm))
    else:
        print(f"Técnica de decodificação {technique} não reconhecida.")

    return results
def run_watermark_attack_suite(
    original_watermarked_image_path: str,
    original_cover_image_path: str,
    secret_message: str,
    decoder_instance, # Can be TrustMark, DWT_DCT_Watermark, invisible-watermark decoder
    technique: int = 2,
    display_images: bool = False,
    len_wm : int = 8,
) -> list:
    """
    Runs a suite of common image processing attacks on a watermarked image
    and evaluates watermark robustness.

    Args:
        original_watermarked_image_path (str): Path to the watermarked image.
        original_cover_image_path (str): Path to the original (unwatermarked) cover image.
                                          Used for comparison metrics or non-blind decoding.
        secret_message (str): The original secret message embedded in the watermark.
        decoder_instance: An instance of the watermarking decoder class
                          (e.g., TrustMark, DWT_DCT_Watermark, invisible-watermark.WatermarkDecoder).
        display_images (bool): If True, attempts to display images after each attack.
                               Requires a display backend (e.g., matplotlib, cv2.imshow).

    Returns:
        list: A list of dictionaries, where each dictionary contains the results
              for a specific attack.
    """
    all_attack_results = []

    # --- Configuration ---
    # The paths and secret are now arguments to the function.

    if not os.path.exists(original_watermarked_image_path):
        print(f"'{original_watermarked_image_path}' não encontrado.")
        print("Por favor, execute o script de embedding para criar a imagem watermarked primeiro.")
        sys.exit(1) # Exit if the necessary watermarked image doesn't exist

    print(f"\nIniciando testes de ataque na imagem: {original_watermarked_image_path}")
    print(f"Segredo Original para verificação: '{secret_message}'")

    if display_images:
        display_image_if_exists(original_watermarked_image_path, "Imagem Original com Marca d'Água")

    # --- Run various attacks ---

    # Resize Attacks
    all_attack_results.append(run_attack_test(
        attack_name = "Redimensionar para 90%", attack_function = attack_resize, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_resized_90.jpg', percentage=25,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm ,
    ))

    all_attack_results.append(run_attack_test(
        attack_name = "Redimensionar para 25%", attack_function = attack_resize, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_resized_25.jpg', percentage=25,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm ,
    ))
    all_attack_results.append(run_attack_test(
        attack_name = "Redimensionar para 50%", attack_function = attack_resize, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_resized_50.jpg', percentage=50, secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm ,
    ))
    all_attack_results.append(run_attack_test(
        attack_name = "Redimensionar para 200%", attack_function =attack_resize, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_resized_200.jpg', percentage=200 ,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))

    # Rotate Attacks
    all_attack_results.append(run_attack_test(
        attack_name = "Rotacionar 15 graus",attack_function =attack_rotate, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_rotated_15.jpg', angle=15,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        attack_name ="Rotacionar 30 graus", attack_function =attack_rotate, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_rotated_30.jpg', angle=30,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))

    # Cut/Crop Attacks
    all_attack_results.append(run_attack_test(
        attack_name ="Cortar (Crop) 10% das bordas", attack_function =attack_cut, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_cut_15.jpg', percentage=15,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        attack_name ="Cortar (Crop) 15% das bordas", attack_function =attack_cut, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_cut_30.jpg', percentage=30,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))

    # Black Border Attack
    all_attack_results.append(run_attack_test(
        attack_name = "Adicionar Borda Preta (10%)", attack_function =attack_black_border,  input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_black_border_10.jpg', border_size_percentage=10,secret=SECRET_MESSAGE, decoder=decoder_instance ,len_wm =len_wm,
    ))
    # Color Change Attacks
    all_attack_results.append(run_attack_test(
        attack_name = "Converter para Escala de Cinza", attack_function =attack_change_colors_grayscale,  input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_grayscale.jpg',secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        attack_name ="Inverter Cores", attack_function =attack_change_colors_invert, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_inverted_colors.jpg',secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    # --- Ataques de Compressão ---
    print("\n--- Iniciando Testes de Ataque de Compressão ---")

    # TIFF (Lossless, using 'tiff_deflate' which is ZIP compression)
    all_attack_results.append(run_attack_test(
         attack_name ="Compressão TIFF (Deflate)", attack_function =attack_compress_tiff,  input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
         output_image_suffix='_tiff_deflate.tiff', compression='tiff_deflate',secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    # TIFF (Lossless, LZW)
    all_attack_results.append(run_attack_test(
        attack_name ="Compressão TIFF (LZW)", attack_function =attack_compress_tiff, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_tiff_lzw.tiff', compression='tiff_lzw',secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))

    # JPEG (Lossy)
    all_attack_results.append(run_attack_test(
        attack_name ="Compressão JPEG (Qualidade 90)", attack_function =attack_compress_jpeg, input_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_jpeg_q90.jpg', quality=90,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        "Compressão JPEG (Qualidade 70)", attack_compress_jpeg, ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_jpeg_q70.jpg', quality=70,secret=SECRET_MESSAGE, decoder=decoder_instance,len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        "Compressão JPEG (Qualidade 50)", attack_compress_jpeg, ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_jpeg_q50.jpg', quality=50,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))

    # JPEG2000 (Lossy by rate)
    all_attack_results.append(run_attack_test(
        "Compressão JPEG2000 (Taxa 5:1)", attack_compress_jpeg2000, ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_jp2_rate5.jp2', quality_mode='rates',secret=SECRET_MESSAGE, decoder=decoder_instance , len_wm =len_wm,
    ))
    all_attack_results.append(run_attack_test(
        "Compressão JPEG2000 (Taxa 10:1)", attack_compress_jpeg2000, ORIGINAL_WATERMARKED_IMAGE_PATH,
        output_image_suffix='_jp2_rate10.jp2', quality_mode='rates', quality=10,secret=SECRET_MESSAGE, decoder=decoder_instance, len_wm =len_wm,
    ))
    # --- Summarize Results ---
    print("\n\n=============== Sumário dos Resultados dos Ataques ===============")
    print(f"{'Ataque':<35} | {'Sucesso':<10} | {'WM Presente':<13} | {'Segredo Corresponde':<20} | {'Segredo Extraído':<30}")
    print("-" * 120)

    for result in all_attack_results:
        attack_name = result['attack_name']
        attack_success = "Sim" if result['attack_successful'] else "Não"
        wm_present = "Sim" if result['watermark_present'] else "Não"
        secret_match = "Sim" if result['secret_match'] else "Não"
        detected_secret = result['detected_secret'] if result['detected_secret'] is not None else "N/A"

        # Truncate secret if too long for display
        if detected_secret != "N/A" and len(str(detected_secret)) > 28:
            detected_secret = str(detected_secret)[:25] + "..."

        print(f"{attack_name:<35} | {attack_success:<10} | {wm_present:<13} | {secret_match:<20} | {detected_secret:<30}")

    print("-" * 120)
    print("Fim dos testes de ataque.")



    return all_attack_results

Is CUDA available?: True
GPU Name: Tesla T4
Total VRAM: 14.74 GB
Using device: cuda


In [ ]:
from blind_watermark import WaterMark
import cv2
import os

def decode_and_evaluate_watermark(
    image_path: str,
    original_secret_message: str,
    # The 'blind_watermark' library's WaterMark class handles its own algorithm internally.
    # So, 'algorithm' parameter is less relevant here unless you customize WaterMark's internals.
    # For simplicity, we'll keep it but it won't directly affect WaterMark's behavior.
    algorithm: str = 'dwtDct' ,
    len_wm: int = 8
) -> dict:
    """
    Decodes a watermark from a watermarked image using blind_watermark.WaterMark
    and evaluates if it matches the original secret.

    Args:
        image_path (str): Path to the watermarked image.
        original_secret_message (str): The original secret used during encoding.
        algorithm (str, optional): This parameter is less relevant for 'blind_watermark'
                                   as its algorithm is inherent to the WaterMark class.
                                   Defaults to 'dwtDct'.

    Returns:
        dict: A dictionary with decoding results:
            - 'image_path': Path to the evaluated image.
            - 'watermark_present': True if something was decoded.
            - 'detected_secret': The decoded message or None.
            - 'secret_match': True if the decoded message matches the original.
            - 'error': Optional error message, if any.
    """
    results = {
        'image_path': image_path,
        'watermark_present': False,
        'detected_secret': None,
        'secret_match': False,
        'error': None
    }

    try:
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image not found at: {image_path}")

        # To extract, we need to know the length of the watermark bit string.
        # This is a crucial piece of information for 'blind_watermark' extraction.
        # If you don't store it, you might need to guess or try different lengths.
        # For this example, we'll assume `len_wm` was obtained during embedding.
        # In a real scenario, you'd need to embed this length or have a fixed length.

        # Calculate the expected length of the watermark bit (8 bits per character for string)
        start_decode_time = time.time()
        bwm1 = WaterMark(password_img=1, password_wm=1)
        wm_extract = bwm1.extract(image_path, wm_shape=len_wm, mode='str')
        print(wm_extract)
        end_decode_time = time.time()
        decode_duration = end_decode_time - start_decode_time
        print(f"Tempo de decodificação: {decode_duration} segundos")

        if wm_extract is not None and len(wm_extract) > 0:
            results['watermark_present'] = True
            results['detected_secret'] = wm_extract

            if wm_extract == original_secret_message:
                results['secret_match'] = True
                print(f"Watermark decoded from '{image_path}' and MATCHES the original secret: '{wm_extract}'")
            else:
                print(f"Watermark decoded from '{image_path}' but DOES NOT MATCH the original secret.")
                print(f"Detected: '{wm_extract}', Original: '{original_secret_message}'")
        else:
            print(f"No watermark detected in '{image_path}'.")

    except FileNotFoundError as e:
        results['error'] = str(e)
        print(f"Error: {e}")
    except Exception as e:
        results['error'] = f"An unexpected error occurred: {e}"
        print(f"An unexpected error occurred: {e}")

    return results


def decode_and_evaluate_imwatermark(
    image_path: str,
    original_secret_message: str,
    algorithm: str = 'dwtDct'
) -> dict:
    """
    Decodes a watermark from a watermarked image and evaluates if it matches the original secret.

    Args:
        image_path (str): Path to the watermarked image (BGR format).
        original_secret_message (str): The original secret used during encoding.
        algorithm (str, optional): The watermarking algorithm used. Defaults to 'dwtDct'.

    Returns:
        dict: A dictionary with decoding results:
            - 'image_path': Path to the evaluated image.
            - 'watermark_present': True if something was decoded.
            - 'detected_secret': The decoded message or None.
            - 'secret_match': True if the decoded message matches the original.
            - 'error': Optional error message, if any.
    """
    results = {
        'image_path': image_path,
        'watermark_present': False,
        'detected_secret': None,
        'secret_match': False
    }

    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Could not load image from: {image_path}")

        # Prepare decoder: expect the exact bit length of the original secret
        expected_bits = 8 * len(original_secret_message)
        decoder = WatermarkDecoder('bytes', expected_bits)
        decoded_bytes = decoder.decode(image, algorithm)

        if decoded_bytes:
            decoded_message = decoded_bytes.decode('utf-8', errors='ignore')
            results['watermark_present'] = True
            results['detected_secret'] = decoded_message

            if decoded_message == original_secret_message:
                results['secret_match'] = True
                print("Watermark decoded and MATCHES the original secret.")
            else:
                print("Watermark decoded but DOES NOT MATCH the original secret.")
        else:
            print(" No watermark detected.")

    except Exception as e:
        results['error'] = str(e)
        print(f" Error during decoding: {e}")

    return results



In [ ]:
num_cpu_cores = os.cpu_count()


In [ ]:
num_cpu_cores

2

In [ ]:
from blind_watermark import WaterMark
import time

start_encode_time = time.time()
bwm1 = WaterMark(password_img=1, password_wm=1,processes=num_cpu_cores)
bwm1.read_img('ufo_240.jpg')
wm = 'f46248768'

bwm1.read_wm(wm, mode='str')
bwm1.embed('embedded.jpg')
len_wm = len(bwm1.wm_bit)
end_encode_time = time.time()
encode_duration = end_encode_time - start_encode_time
print(f"Tempo de codificação: {encode_duration} segundos")
print('Put down the length of wm_bit {len_wm}'.format(len_wm=len_wm))


Tempo de codificação: 1.2524774074554443 segundos
Put down the length of wm_bit 71


In [ ]:
import torch
import os

print("### ESPECIFICAÇÃO DA MÁQUINA (Google Colab) ###\n")

# --- CPU ---
print("--- Processador (CPU) ---")
# O comando !lscpu mostra detalhes da CPU
# Extrai a linha do nome do modelo para uma saída mais limpa
cpu_info = !lscpu | grep 'Model name'
print(cpu_info[0].strip())
# O comando !nproc mostra o número de núcleos
print(f"Número de núcleos: {os.cpu_count()}\n")

# --- Memória RAM ---
print("--- Memória RAM ---")
# O comando !cat /proc/meminfo mostra detalhes da memória
# Extrai a linha da memória total (MemTotal) e converte para GB
mem_info = !cat /proc/meminfo | grep 'MemTotal'
mem_total_kb = int(mem_info[0].split()[1])
mem_total_gb = mem_total_kb / (1024**2)
print(f"Total: {mem_total_gb:.2f} GB\n")

# --- Placa de Vídeo (GPU) ---
print("--- Placa de Vídeo (GPU) ---")
if torch.cuda.is_available():
    # Obtém o nome da GPU
    gpu_name = torch.cuda.get_device_name(0)
    # Obtém a memória VRAM total da GPU em bytes e converte para GB
    gpu_vram_bytes = torch.cuda.get_device_properties(0).total_memory
    gpu_vram_gb = gpu_vram_bytes / (1024**3)

    print(f"Nome da GPU: {gpu_name}")
    print(f"Total VRAM: {gpu_vram_gb:.2f} GB")
    print(f"Dispositivo em uso: cuda")
else:
    print("GPU não detectada. Dispositivo em uso: cpu\n")

# --- Sistema Operacional ---
print("\n--- Sistema Operacional ---")
# O comando !lsb_release -a mostra os detalhes do SO
!lsb_release -a

print("\n-------------------------------------------")

### ESPECIFICAÇÃO DA MÁQUINA (Google Colab) ###

--- Processador (CPU) ---
Model name:                           Intel(R) Xeon(R) CPU @ 2.30GHz
Número de núcleos: 2

--- Memória RAM ---
Total: 12.67 GB

--- Placa de Vídeo (GPU) ---
Nome da GPU: Tesla T4
Total VRAM: 14.74 GB
Dispositivo em uso: cuda

--- Sistema Operacional ---
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy

-------------------------------------------


In [ ]:
similarity_result =[]
r = get_similarity_metrics("blind watermark", 'ufo_240.jpg', 'embedded.jpg')
similarity_result.append(r)
print_metrics_table()


--- Calculando métricas para: blind watermark entre 'ufo_240.jpg' e 'embedded.jpg' ---
  MSE: 29.1674
  SSIM: 0.9794
  FSIM: N/A (Not available in this setup)
  VIF: 0.7231
------------------------------------------------------------

--- Interpretação das Métricas Perceptuais ---
Essas métricas buscam correlacionar-se melhor com a percepção humana da qualidade e similaridade da imagem:
- **SSIM**: Excelente para capturar a estrutura, contraste e luminância.
- **FSIM**: Foca em características de baixo nível que o sistema visual humano é bom em detectar (gradientes de fase e magnitude).
- **VIFp**: Modela como a informação é extraída pelo sistema visual humano, avaliando a fidelidade da informação visual.
--- Entendendo as Métricas de Similaridade de Imagem ---

Ao avaliar a similaridade entre duas imagens, especialmente após processos como compressão ou esteganografia, usamos métricas para quantificar o quão parecidas elas são. Algumas dessas métricas focam na diferença exata de pixe

In [ ]:
# Now use the new function to decode and evaluate
print("\n--- Decoding and Evaluating with decode_and_evaluate_watermark ---")
original_secret =wm
evaluation_results = decode_and_evaluate_watermark(
    image_path='embedded.jpg',
    original_secret_message=original_secret, len_wm =len_wm
)

print("\nEvaluation Results:")
for key, value in evaluation_results.items():
    print(f"  {key}: {value}")

# Example with a non-existent image to show error handling
print("\n--- Decoding non-existent image ---")



--- Decoding and Evaluating with decode_and_evaluate_watermark ---
f46248768
Tempo de decodificação: 0.7259409427642822 segundos
Watermark decoded from 'embedded.jpg' and MATCHES the original secret: 'f46248768'

Evaluation Results:
  image_path: embedded.jpg
  watermark_present: True
  detected_secret: f46248768
  secret_match: True
  error: None

--- Decoding non-existent image ---


In [ ]:
ORIGINAL_WATERMARKED_IMAGE_PATH = 'embedded.jpg'
ORIGINAL_COVER_IMAGE_PATH = 'ufo_240.jpg'# Needed if TrustMark decode is non-blind or for comparison
SECRET_MESSAGE = wm # The secret you embedded

all_attack_results = run_watermark_attack_suite(
        original_watermarked_image_path=ORIGINAL_WATERMARKED_IMAGE_PATH,
        original_cover_image_path=ORIGINAL_COVER_IMAGE_PATH,
        secret_message=SECRET_MESSAGE,
        decoder_instance=None,
        display_images=False, # Set to True if you want to see images (requires display setup)
        technique=4,
        len_wm =len_wm,
    )


Iniciando testes de ataque na imagem: embedded.jpg
Segredo Original para verificação: 'f46248768'

===== Testando Ataque: Redimensionar para 90% =====
Aplicando ataque e salvando em: embedded_resized_90.jpg
--- Tentando decodificar a marca d'água ---
�8C� ��8
Tempo de decodificação: 0.042328596115112305 segundos
Watermark decoded from 'embedded_resized_90.jpg' but DOES NOT MATCH the original secret.
Detected: '�8C� ��8', Original: 'f46248768'

===== Testando Ataque: Redimensionar para 25% =====
Aplicando ataque e salvando em: embedded_resized_25.jpg
--- Tentando decodificar a marca d'água ---
�8C� ��8
Tempo de decodificação: 0.039670467376708984 segundos
Watermark decoded from 'embedded_resized_25.jpg' but DOES NOT MATCH the original secret.
Detected: '�8C� ��8', Original: 'f46248768'

===== Testando Ataque: Redimensionar para 50% =====
Aplicando ataque e salvando em: embedded_resized_50.jpg
--- Tentando decodificar a marca d'água ---
g�]q�6_�
Tempo de decodificação: 0.1744656562

In [ ]:
decode_and_evaluate_watermark(image_path='embedded_resized_25.png', original_secret_message=SECRET_MESSAGE,len_wm = len_wm)

Error: Image not found at: embedded_resized_25.png


{'image_path': 'embedded_resized_25.png',
 'watermark_present': False,
 'detected_secret': None,
 'secret_match': False,
 'error': 'Image not found at: embedded_resized_25.png'}